Имеются данные adult.csv (см. в материалах к занятию)

Целевой переменной является уровень дохода income (крайний правый столбец).

Описание признаков можно найти по ссылке http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html

Вам необходимо построить модель **логистической регрессии**, которая предсказывает уровень дохода человека. При возможности попробуйте улучшить точность предсказаний (метод score) с помощью перебора признаков.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('adult.csv')

In [3]:
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


**посмотрим на распределение наших ответов**

In [4]:
df['income'].value_counts(normalize=True)*100

<=50K    76.071823
>50K     23.928177
Name: income, dtype: float64

**теперь посмотрим на данные, нужно ли нам работать с пропусками и т.д**

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
age                48842 non-null int64
workclass          48842 non-null object
fnlwgt             48842 non-null int64
education          48842 non-null object
educational-num    48842 non-null int64
marital-status     48842 non-null object
occupation         48842 non-null object
relationship       48842 non-null object
race               48842 non-null object
gender             48842 non-null object
capital-gain       48842 non-null int64
capital-loss       48842 non-null int64
hours-per-week     48842 non-null int64
native-country     48842 non-null object
income             48842 non-null object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [7]:
# попробуем выбрать какие-нибудь признаки
selectedColumns = df[ [ 'hours-per-week', 'gender', 'race','relationship','occupation','marital-status','income'] ]

# зашифруем категориальные переменные с помощью get_dummies
# переведем их в значения 0 и 1, добавив столбцы с соответствующими названиями
X = pd.get_dummies( selectedColumns, columns = [ 'gender', 'race','relationship','occupation','marital-status' ] )

# столбец income является целевой переменной, удаляем его из X
del X['income']
X.head()

,hours-per-week,gender_Female,gender_Male,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,relationship_Husband,relationship_Not-in-family,...,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,marital-status_Divorced,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,marital-status_Separated,marital-status_Widowed
0,40,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,0,1,0,0,0,0,1,1,0,...,0,0,0,0,0,1,0,0,0,0
2,40,0,1,0,0,0,0,1,1,0,...,0,0,0,0,0,1,0,0,0,0
3,40,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,30,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [8]:
# целевая переменная (столбец income) снова является категориальной
# переведем значения столбца в числа, оставив один столбец

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit( df['income'] )
le.classes_

array(['<=50K', '>50K'], dtype=object)

In [9]:
y = pd.Series( data = le.transform( df['income'] ) )
y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int32

**Теперь разделим наши данные на train/test**

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
model = LogisticRegression()

In [12]:
# обучаем модель

model.fit( X_train, y_train )
predictions = model.predict_proba( X_test )

c:\users\sladk\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [13]:
model.score(X_test, y_test)

0.8264919643771113

**В прошлый раз мы не брали возраст, в этот раз добавим его**

In [14]:
# попробуем выбрать какие-нибудь признаки
selectedColumns = df[ [ 'age','hours-per-week', 'gender', 'race','relationship','occupation','marital-status','income'] ]

# зашифруем категориальные переменные с помощью get_dummies
# переведем их в значения 0 и 1, добавив столбцы с соответствующими названиями
X = pd.get_dummies( selectedColumns, columns = [ 'gender', 'race','relationship','occupation','marital-status' ] )

# столбец income является целевой переменной, удаляем его из X
del X['income']
# заново делим выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
model.fit( X_train, y_train )
predictions = model.predict_proba( X_test )
model.score(X_test, y_test)

c:\users\sladk\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8275156106049749

**в итоге, добавив в обучающую выборку, мы улучшили качество модели**